In [1]:
import os, tempfile
from builtins import zip
from builtins import range
import unittest2
import testdata
from gaia2 import transform
from gaia2 import DataSet
from gaia2 import Point
from gaia2 import EnumType
from gaia2 import cvar
from gaia2 import MetricFactory
from gaia2 import AnalyzerFactory
from gaia2 import View
cvar.verbose = True

In [3]:
"""
#def applyToDataSet(self, dataset):
#        """
#        DataSet
#        * gaia2::Transformation::applyToDataSet(const DataSet *dataset) const

#       Apply the current transformation with its parameters to the DataSet.

#        """
#        return _gaia2.Transformation_applyToDataSet(self, dataset)

# complete transformation in just one function call
"""
def transform(dataset, transfoName, params = {}):
    analyzer = AnalyzerFactory.create(transfoName, params)
    return analyzer.analyze(dataset).applyToDataSet(dataset)
"""    
class TestDataSet(unittest2.TestCase):

    def setUp(self):
        verbose = cvar.verbose
       
    def tearDown(self):
        testdata.resetSettings()

    def testDoubleLoad(self):
        ds1 = DataSet()
        ds1.load(testdata.TEST_DATABASE)
        ds1.load(testdata.TEST_DATABASE)

        ds2 = DataSet()
        ds2.load(testdata.TEST_DATABASE)

        self.assertEqual(ds1.size(), ds2.size())
        for p1, p2 in zip(ds1.points(), ds2.points()):
            self.assertEqual(p1, p2)

    def testLinkedDataSets(self):
        d = testdata.createSimpleDataSet()
        d2 = transform(d, 'Select', { 'descriptorNames': [ 'a*', 'b*' ] })

        d2.setReferenceDataSet(d)

        p = Point()
        p.setName('ptest')
        p.setLayout(d.originalLayout())
        p['a.1'] = 23

        d.addPoint(p)

        self.assertEqual(d2.point('ptest').value('a.1'), 23)
        #print (d2.point('ptest').value)
        self.assertRaises(Exception, d2.point('ptest').value('c'))

        d2.removePoint('ptest')
        self.assertRaises(Exception, d.point, 'ptest')

    def testDeleteLinkedDataSets(self):
        ds = testdata.createSimpleDataSet()
        ds.save('ds.db')
        ds2 = transform(ds, 'select', { 'descriptorNames': [ 'a*', 'b*' ] })
        ds2.save('ds2.db')
        ds2.setReferenceDataSet(ds)

        del ds2
        del ds
        
    def testLinkedViews(self):
        ds = testdata.createSimpleDataSet()
        ds.point('p')['a.1'] = 23.0

        for i in range(5):
            p = Point()
            p.setName('p%d' % i)
            p.setLayout(ds.originalLayout())
            p['a.1'] = float(i)
            ds.addPoint(p)

        ds = transform(ds, 'fixlength')
        dist = MetricFactory.create('euclidean', ds.layout())
        v = View(ds)

        query = Point(ds.point('p'))
        query['a.1'] = 23.0

        self.assertEqual((('p', 0.), ('p4', 19.), ('p3', 20.), ('p2', 21.)),
                         v.nnSearch(query, dist).get(4))

        ds.removePoint('p')
        self.assertEqual((('p4', 19.), ('p3', 20.), ('p2', 21.), ('p1', 22.)),
                         v.nnSearch(query, dist).get(4))

        ds.removePoint('p3')
        self.assertEqual((('p4', 19.), ('p2', 21.), ('p1', 22.), ('p0', 23.)),
                         v.nnSearch(query, dist).get(4))

        # get original query point to add in dataset
        testdata.resetSettings()
        dsq = testdata.createSimpleDataSet()
        orig_query = Point(dsq.point('p'))
        orig_query['a.1'] = 23.0

        ds.addPoint(orig_query)
        self.assertEqual((('p', 0.), ('p4', 19.), ('p2', 21.), ('p1', 22.)),
                         v.nnSearch(query, dist).get(4))


    def testHistory(self):
        # add a point in a transformed dataset
        d = testdata.createSimpleDataSet()
        ds = transform(d, 'select', { 'descriptorNames': [ 'a*', 'b*' ] })

        p = Point()
        p.setName('ptest')
        p.setLayout(testdata.createSimpleLayout())

        ds.addPoint(p)

    def testCollectionMultipleAdd(self):
        d = testdata.loadTestDB()
        d1 = DataSet()
        d2 = DataSet()
        d3 = DataSet()

        # add them one by one
        for p in d.points():
            d1.addPoint(p)

        # add them all at once
        d2.addPoints(d)

        # add them all at once using a list of points
        d3.addPoints([ p for p in d.points() ])

        self.assertEquals(d1.size(), d2.size())
        for p1, p2 in zip(d1.points(), d2.points()):
            self.assertEqual(p1, p2)

        self.assertEquals(d1.size(), d3.size())
        for p1, p3 in zip(d1.points(), d3.points()):
            self.assertEqual(p1, p3)

    def createSequentialDataSet(self):
        # used in testRemovePoints
        layout = testdata.createSimpleLayout()
        ds = DataSet()
        p = Point()
        p.setLayout(layout)
        for name in [ '0', '1', '2', '3', '4', '5' ]:
            p.setName(name)
            ds.addPoint(p)

        return ds

    def testCollectionRemovePoints(self):
        ds = self.createSequentialDataSet()
        ds.removePoints([])
        self.assertEquals(ds[0].name(), '0')
        self.assertEquals(ds[1].name(), '1')
        self.assertEquals(ds[2].name(), '2')
        self.assertEquals(ds[3].name(), '3')
        self.assertEquals(ds[4].name(), '4')
        self.assertEquals(ds[5].name(), '5')

        ds = self.createSequentialDataSet()
        ds.removePoints([ '3', '4', '5' ])
        self.assertEquals(ds[0].name(), '0')
        self.assertEquals(ds[1].name(), '1')
        self.assertEquals(ds[2].name(), '2')

        ds = self.createSequentialDataSet()
        ds.removePoints([ '1', '2' ])
        self.assertEquals(ds[0].name(), '0')
        self.assertEquals(ds[1].name(), '3')
        self.assertEquals(ds[2].name(), '4')
        self.assertEquals(ds[3].name(), '5')

        ds = self.createSequentialDataSet()
        ds.removePoints([ '1', '3' ])
        self.assertEquals(ds[0].name(), '0')
        self.assertEquals(ds[1].name(), '2')
        self.assertEquals(ds[2].name(), '4')
        self.assertEquals(ds[3].name(), '5')

        ds = self.createSequentialDataSet()
        ds.removePoints([ '2', '5' ])
        self.assertEquals(ds[0].name(), '0')
        self.assertEquals(ds[1].name(), '1')
        self.assertEquals(ds[2].name(), '3')
        self.assertEquals(ds[3].name(), '4')


    def testAppendDataSet(self):
        ds = testdata.loadTestDB()
        ds2 = self.createSequentialDataSet()

        # check homogeneous collections
        self.assertRaises(Exception, ds.appendDataSet, ds2)

        # check no duplicates
        self.assertRaises(Exception, ds2.appendDataSet, ds2)

        # check normal operation of appendCollection
        ppoints = [ Point(p) for p in ds2.points() ]
        for p in ppoints:
            p.setName('p' + p.name())

        ds3 = DataSet()
        ds3.addPoints(ppoints)
        ds3.appendDataSet(ds2)

        for p in ds3.points():
            if p.name()[0] == 'p':
                self.assert_(p == ds3.point(p.name()[1:]))
            else:
                self.assert_(p == ds3.point('p' + p.name()))

    def testAppendDataSetWithEnums(self):
        ds = DataSet()
        ds.setReferenceDataSet(None)
        ds.load('data/pv.yml.124.ds.essentia1.0.6_ref.db')

        enumValues = {}
        enumDescs = ds.layout().descriptorNames(EnumType)

        for p in ds.points():
            for desc in enumDescs:
                enumValues[(p.name(), desc)] = p[desc]

        ds2 = DataSet()
        ds2.load('data/pv.yml.125.ds.essentia1.0.6_ref.db')

        for p in ds2.points():
            for desc in enumDescs:
                enumValues[(p.name(), desc)] = p[desc]

        ds.appendDataSet(ds2)

        # assert all values in enum values are ok
        for (pname, desc), value in list(enumValues.items()):
            self.assertEqual(value, ds.point(pname)[desc])

        ds3 = DataSet()
        ds3.load('data/pv.yml.126.ds.essentia1.0.6_ref.db')

        for p in ds3.points():
            for desc in enumDescs:
                enumValues[(p.name(), desc)] = p[desc]

        ds.appendDataSet(ds3)

        # check all values are all still the same
        for (pname, desc), value in list(enumValues.items()):
            self.assertEqual(value, ds.point(pname)[desc])

        (tmpFile, tmpName) = tempfile.mkstemp()
        os.close(tmpFile)
        ds.save(tmpName)
        ds.load(tmpName)

        for (pname, desc), value in list(enumValues.items()):
            self.assertEqual(value, ds.point(pname)[desc])

        os.remove(tmpName)


    def testFixLength(self):
        testdata.useFixedLength = True
        self.testLinkedDataSets()
        self.testLinkedViews()
        self.testHistory()
        self.testCollectionMultipleAdd()

    def testEnumerate(self):
        testdata.useEnumerate = True
        self.testLinkedDataSets()
        self.testLinkedViews()
        self.testHistory()
        self.testCollectionMultipleAdd()

    def testEnumerateFixLength(self):
        testdata.useEnumerate = True
        self.testFixLength()



In [4]:
suite = unittest2.TestLoader().loadTestsFromTestCase(TestDataSet)
if __name__ == '__main__':
    unittest2.TextTestRunner(verbosity=2).run(suite)

testAppendDataSet (__main__.TestDataSet) ... ok
testAppendDataSetWithEnums (__main__.TestDataSet) ... ok
testCollectionMultipleAdd (__main__.TestDataSet) ... ok
testCollectionRemovePoints (__main__.TestDataSet) ... ok
testDeleteLinkedDataSets (__main__.TestDataSet) ... ok
testDoubleLoad (__main__.TestDataSet) ... ok
testEnumerate (__main__.TestDataSet) ... ERROR
testEnumerateFixLength (__main__.TestDataSet) ... ERROR
testFixLength (__main__.TestDataSet) ... ok
testHistory (__main__.TestDataSet) ... ok
testLinkedDataSets (__main__.TestDataSet) ... ok
testLinkedViews (__main__.TestDataSet) ... ok

ERROR: testEnumerate (__main__.TestDataSet)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-3-cf90d3ea46b5>", line 276, in testEnumerate
  File "<ipython-input-3-cf90d3ea46b5>", line 39, in testLinkedDataSets
  File "/home/gordon/gaia/test/unittest/testdata.py", line 96, in createSimpleDataSet
    ds = enumerateStr

In [ ]:
"""from gaia2 import *
import _gaia2
import gaia2
def _swig_setattr_nondynamic(self, class_type, name, value, static=1):
    if (name == "thisown"):
        return self.this.own(value)
    if (name == "this"):
        if type(value).__name__ == 'SwigPyObject':
            self.__dict__[name] = value
            return
    method = class_type.__swig_setmethods__.get(name, None)
    if method:
        return method(self, value)
    if (not static):
        if _newclass:
            object.__setattr__(self, name, value)
        else:
            self.__dict__[name] = value
    else:
        raise AttributeError("You cannot add attributes to %s" % self)


def _swig_setattr(self, class_type, name, value):
    return _swig_setattr_nondynamic(self, class_type, name, value, 0)


def _swig_getattr(self, class_type, name):
    if (name == "thisown"):
        return self.this.own()
    method = class_type.__swig_getmethods__.get(name, None)
    if method:
        return method(self)
    raise AttributeError("'%s' object has no attribute '%s'" % (class_type.__name__, name))


def _swig_repr(self):
    try:
        strthis = "proxy of " + self.this.__repr__()
    except __builtin__.Exception:
        strthis = ""
    return "<%s.%s; %s >" % (self.__class__.__module__, self.__class__.__name__, strthis,)

try:
    _object = object
    _newclass = 1
except __builtin__.Exception:
    class _object:
        pass
    _newclass = 0

class ParameterMap(_object):
    __swig_setmethods__ = {}
    __setattr__ = lambda self, name, value: _swig_setattr(self, ParameterMap, name, value)
    __swig_getmethods__ = {}
    __getattr__ = lambda self, name: _swig_getattr(self, ParameterMap, name)
    __repr__ = _swig_repr

    def __init__(self):
        this = _gaia2.new_ParameterMap()
        try:
            self.this.append(this)
        except __builtin__.Exception:
            self.this = this
    __swig_destroy__ = _gaia2.delete_ParameterMap
    def __del__(self):
        return None
ParameterMap_swigregister = _gaia2.ParameterMap_swigregister
ParameterMap_swigregister(ParameterMap)

class Parameter(_object):
    __swig_setmethods__ = {}
    __setattr__ = lambda self, name, value: _swig_setattr(self, Parameter, name, value)
    __swig_getmethods__ = {}
    __getattr__ = lambda self, name: _swig_getattr(self, Parameter, name)
    __repr__ = _swig_repr

    def toArrayReal(self):
        return _gaia2.Parameter_toArrayReal(self)

    def toRealDescriptor(self):
        return _gaia2.Parameter_toRealDescriptor(self)

    def toStringDescriptor(self):
        return _gaia2.Parameter_toStringDescriptor(self)

    def toParameterMap(self):
        return _gaia2.Parameter_toParameterMap(self)

    def toMatrix(self):
        return _gaia2.Parameter_toMatrix(self)

    def toTransformation(self):
        return _gaia2.Parameter_toTransformation(self)

    def toHistory(self):
        return _gaia2.Parameter_toHistory(self)

    def toCoords(self):
        return _gaia2.Parameter_toCoords(self)

    def __init__(self, *args):
        this = _gaia2.new_Parameter(*args)
        try:
            self.this.append(this)
        except __builtin__.Exception:
            self.this = this

    def toPointLayout(self):
        return _gaia2.Parameter_toPointLayout(self)
    __swig_destroy__ = _gaia2.delete_Parameter
    def __del__(self):
        return None
Parameter_swigregister = _gaia2.Parameter_swigregister
Parameter_swigregister(Parameter)


#from gaia2 import AnalyzerFactory
from past.builtins import basestring
def transform(dataset, transfoName, params = {}):
    analyzer = AnalyzerFactory.create(transfoName, params)
    return analyzer.analyze(dataset).applyToDataSet(dataset)

def _AnalyzerFactory_create(name, params = {}):
    result = gaia2.AnalyzerFactory_create(name, pmap(params))
    try:
        result.__doc__ = gaia2_docDict['analyzer'][name.lower()]
    except: pass
    return result

AnalyzerFactory.create = staticmethod(_AnalyzerFactory_create)
#import ast
def pmap(param_map):
    print ("type(param_map): ", type(param_map))
    print ("param_map['descriptorNames']: ", param_map['descriptorNames'])
    if isinstance(param_map, ParameterMap):
        return param_map

    if not isinstance(param_map, dict):
        raise TypeError('Argument is not a map, it is a %s' % type(param_map))

    pm = ParameterMap()
    for key, value in param_map.items():
        if not isinstance(key, basestring):
            raise TypeError('A key in the map is not a string; can not convert it')

        # until python 3, everything should always be utf-8 encoded bytestrings
        if isinstance(key, unicode):
            key = key.encode('utf-8')
        if isinstance(value, unicode):
            value = value.encode('utf-8')

        if isinstance(value, (dict, ParameterMap)): # if value is a python map, convert it to ParameterMap
            pm.setParameterMap(key, pmap(value))
        else: # all other types just go inside the map directly
            # pm[key] = value
            pass
    return pm


ParameterMap.setParameterMap = lambda pm, key, value: _gaia2.ParameterMap_setParameterMap(pm, key, pmap(value))"""

In [ ]:
exclude = []
select = []
useFixedLength = False
useEnumerate = True
from gaia2 import PointLayout, transform, DataSet
from gaia2 import StringType, RealType, FixedLength 

def filter(select, exclude):
    return _gaia2.PointLayout_filter(select, exclude)
def enumerateStrings(dataset, exclude =[]):
    sel = (dataset.layout().descriptorNames(StringType))
    print (sel)
    #exclude =['a.1','a.2','b','c','e']
    #print (exclude)
    #select = (dataset.layout().descriptorNames)
    ds = transform(dataset, 'Select', { 'descriptorName': 'd' })
    print (ds.layout()) 
    ds.setReferenceDataSet(dataset)
    print (ds.layout())
    #print (type ('select'))
    return transform(ds, 'Enumerate', { 'descriptorName': 'd'})

# create simple artificial layout
def createSimpleLayout():
    l = PointLayout()
    l.add('a.1', RealType)
    l.add('a.2', RealType)
    l.add('c', RealType)
    l.add('b', RealType)
    l.add('d', StringType)
    l.add('e', RealType, FixedLength, 3)
    return l


# create simple artificial dataset
def createSimpleDataSet():
    global useFixedLength,  useEnumerate
    l = createSimpleLayout()
    ds = DataSet()
    p = Point()
    p.setName('p')
    p.setLayout(l)
    ds.addPoint(p)

    if useFixedLength:
        ds2 = fixLength(ds)

    if useEnumerate:
        ds2 = enumerateStrings(ds)

    return ds2

useEnumerate = True
d = createSimpleDataSet()
#print (dir(gaia2.PointLayout))
#print (exclude)
#exclude =['a.1','a.2','b','c','e']
print(d.layout().descriptorName)
#ds.save('ds.db')
#ds2 = transform(ds, 'select', { 'descriptorNames': [ 'a*', 'b*'] })
#ds2.save('ds2.db')
#ds2.setReferenceDataSet(ds)
#del ds2
#del ds

In [ ]:
def cfg_from_list(cfg_list):
  """Set config keys via list (e.g., from command line)."""
  from ast import literal_eval
  assert len(cfg_list) % 2 == 0
  for k, v in zip(cfg_list[0::2], cfg_list[1::2]):
    key_list = k.split('.')
    d = __C
    for subkey in key_list[:-1]:
      assert subkey in d
      d = d[subkey]
    subkey = key_list[-1]
    assert subkey in d
    try:
      value = literal_eval(v)
    except:
      # handle the case when v is a string literal
      value = v
    assert type(value) == type(d[subkey]), \
      'type {} does not match original type {}'.format(
        type(value), type(d[subkey]))
    d[subkey] = value 

In [ ]:
import testdata
from gaia2 import transform
from gaia2 import Point
import unittest2

#from unittest2.TestCase import assertEqual
#from unittest2.TestCase import assertRaises

d = testdata.createSimpleDataSet()
d2 = transform(d, 'select', { 'descriptorNames': [ 'a*', 'b*' ] })

d2.setReferenceDataSet(d)

p = Point()
p.setName('ptest')
p.setLayout(d.originalLayout())
p['a.1'] = 23

d.addPoint(p)

print (d2.point('ptest').value('a.1'))
#unittest2.TestCase.assertEqual((d2.point('ptest').value('a.1')), 23)
#unittest2.TestCase.assertRaises(Exception, d2.point('ptest').value, 'c')

d.removePoint('ptest')
#unittest2.TestCase.assertRaises(Exception, d.point, 'ptest')

In [ ]:
dir (unittest2)

In [ ]:
dir (unittest2.TestCase)

In [ ]:
from gaia2 import DataSet
dir (DataSet)

In [ ]:
from gaia2 import PointLayout 
dir (PointLayout)

In [ ]:
from gaia2 import PointLayout

In [ ]:
dir (PointLayout)